# AutoML - Train "the best" NLP Multi-Label Text Classification model for a paper abstract dataset.

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

This notebook demonstrates multilabel classification with text data using AutoML NLP.

Scenario: Paper submission systems (such as CMT, OpenReview, etc.) require the users to upload paper titles and paper abstracts and then specify the subject areas their papers best belong to. Creating this model, you can classify/suggest what category a corresponding papers could be best associated with.

Configuration and remote run of AutoML for a multilabel text classification dataset from [Kaggle](https://www.kaggle.com), [arXiv Paper Abstracts](https://www.kaggle.com/datasets/spsayakpaul/arxiv-paper-abstracts).

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient

from azure.ml._constants import AssetTypes
from azure.ml.entities import JobInput

from azure.ml import automl

# from azure.ml.automl import text_classification_multilabel

from pprint import pprint

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
credential = InteractiveBrowserCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

# 2. Data

Since the original dataset is very large, we leverage a subsampled dataset to allow for faster training for the purposes of running this example notebook. To run the full dataset (50K+ samples and 1k+ labels) you might need a GPU instance with larger memory and it may take longer to finish training.

To run the code below, you can first download 'arxiv_data.csv' from [this link](https://www.kaggle.com/spsayakpaul/arxiv-paper-abstracts) and save it under the same directory as this notebook, and then run preprocessing.py to create a subset of the data for training and evaluation, then split in training, validation and test dataset files. However, for your convenience, we already provide it prepared within the several mltable folders.

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

In [ ]:
# MLTable folders
training_mltable_path = "./training-mltable-folder/"
validation_mltable_path = "./validation-mltable-folder/"

# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = JobInput(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = JobInput(
    type=AssetTypes.MLTABLE, path=validation_mltable_path
)

# WITH REMOTE PATH: If available already in the cloud/workspace-blob-store
# my_training_data_input = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my_training_mltable")
# my_validation_data_input = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my_validation_mltable")

# 3. Configure and run the AutoML NLP Text Classification Multilabel training job
In this section we will configure and run the AutoML job, for training the model.    

In [ ]:
# general job parameters
compute_name = "gpu-cluster"
exp_name = "dpv2-nlp-text-classification-multilabel-experiment"

In [ ]:
# Create the AutoML job with the related factory-function.

text_classification_multilabel_job = automl.text_classification_multilabel(
    compute=compute_name,
    # name="dpv2-nlp-text-classification-multilabel-job-02",
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="terms",
    primary_metric="accuracy",
    tags={"my_custom_tag": "My custom value"},
    # These are temporal properties needed in Private Preview
    properties={"_automl_internal_save_mlflow": True},
)

text_classification_multilabel_job.set_limits(timeout=120)

## 2.2 Run the CommandJob
Using the `MLClient` created earlier, we will now run this CommandJob in the workspace.

In [ ]:
# Submit the AutoML job

returned_job = ml_client.jobs.create_or_update(
    text_classification_multilabel_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

# Next Steps
You can see further examples of other AutoML tasks such as Regression, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.